In [1]:
import ee
import folium
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
# Adjust the 'center', 'zoom', and 'geojson' paths as needed.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Function to get a MODIS Land Cover image for a given year.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    return image

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 0,
    'max': 17,
    'palette': [
        '05450a',  # Evergreen Needleleaf Forests (Class 1)
        '086a10',  # Evergreen Broadleaf Forests (Class 2)
        '54a708',  # Deciduous Needleleaf Forests (Class 3)
        '78d203',  # Deciduous Broadleaf Forests (Class 4)
        '009900',  # Mixed Forests (Class 5)
        'c6b044',  # Closed Shrublands (Class 6)
        'dcd159',  # Open Shrublands (Class 7)
        'dade48',  # Woody Savannas (Class 8)
        'fbff13',  # Savannas (Class 9)
        'b6ff05',  # Grasslands (Class 10)
        '27ff87',  # Permanent Wetlands (Class 11)
        'c24f44',  # Croplands (Class 12)
        'a5a5a5',  # Urban and Built-up Lands (Class 13)
        'ff6d4c',  # Cropland/Natural Vegetation Mosaics (Class 14)
        '69fff8',  # Permanent Snow and Ice (Class 15)
        'f9ffa4',  # Barren (Class 16)
        '1c0dff'   # Water Bodies (Class 17)
    ]
}

# Helper function to add Earth Engine layers to a Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Function to add a legend to the map with just the class names.
def add_legend(map_object):
    legend_html = '''
     <div style="
         position: fixed; 
         bottom: 50px; left: 50px; 
         width: 240px; 
         border:2px solid grey; 
         z-index:9999; 
         font-size:12px;
         background-color:white;
         opacity: 0.8;
         padding: 10px;
         ">
         <b>Land Cover Legend</b><br>
         <i style="background:#05450a; width:18px; height:18px; float:left; margin-right:8px;"></i>Evergreen Needleleaf Forests<br>
         <i style="background:#086a10; width:18px; height:18px; float:left; margin-right:8px;"></i>Evergreen Broadleaf Forests<br>
         <i style="background:#54a708; width:18px; height:18px; float:left; margin-right:8px;"></i>Deciduous Needleleaf Forests<br>
         <i style="background:#78d203; width:18px; height:18px; float:left; margin-right:8px;"></i>Deciduous Broadleaf Forests<br>
         <i style="background:#009900; width:18px; height:18px; float:left; margin-right:8px;"></i>Mixed Forests<br>
         <i style="background:#c6b044; width:18px; height:18px; float:left; margin-right:8px;"></i>Closed Shrublands<br>
         <i style="background:#dcd159; width:18px; height:18px; float:left; margin-right:8px;"></i>Open Shrublands<br>
         <i style="background:#dade48; width:18px; height:18px; float:left; margin-right:8px;"></i>Woody Savannas<br>
         <i style="background:#fbff13; width:18px; height:18px; float:left; margin-right:8px;"></i>Savannas<br>
         <i style="background:#b6ff05; width:18px; height:18px; float:left; margin-right:8px;"></i>Grasslands<br>
         <i style="background:#27ff87; width:18px; height:18px; float:left; margin-right:8px;"></i>Permanent Wetlands<br>
         <i style="background:#c24f44; width:18px; height:18px; float:left; margin-right:8px;"></i>Croplands<br>
         <i style="background:#a5a5a5; width:18px; height:18px; float:left; margin-right:8px;"></i>Urban and Built-up Lands<br>
         <i style="background:#ff6d4c; width:18px; height:18px; float:left; margin-right:8px;"></i>Cropland/Natural Vegetation Mosaics<br>
         <i style="background:#69fff8; width:18px; height:18px; float:left; margin-right:8px;"></i>Permanent Snow and Ice<br>
         <i style="background:#f9ffa4; width:18px; height:18px; float:left; margin-right:8px;"></i>Barren<br>
         <i style="background:#1c0dff; width:18px; height:18px; float:left; margin-right:8px;"></i>Water Bodies<br>
     </div>
    '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

# Create ipywidgets for country selection and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Output widget for displaying the map.
output = widgets.Output()

# Function to update the map based on the selected country and year.
def update_map(change):
    # Get current selections.
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create a new Folium map centered on the selected country.
    new_map = folium.Map(location=center, zoom_start=zoom)
    folium.TileLayer('OpenStreetMap').add_to(new_map)
    
    # Add the GeoJSON layer for the country's boundary.
    folium.GeoJson(
        geojson_path,
        name=f"{country.title()} Boundary"
    ).add_to(new_map)
    
    # Retrieve the MODIS image for the selected year.
    image = get_modis_image(year)
    new_map.add_ee_layer(image, vis_params, 'MODIS Land Cover ' + str(year))
    
    # Add the custom legend.
    add_legend(new_map)
    
    # Add layer control to toggle layers.
    new_map.add_child(folium.LayerControl())
    
    # Display the updated map.
    with output:
        output.clear_output()
        display(new_map)

# Observe changes in both widgets.
country_dropdown.observe(update_map, names='value')
year_slider.observe(update_map, names='value')

# Display the initial map.
update_map({'new': country_dropdown.value})
display(widgets.VBox([country_dropdown, year_slider, output]))

/Users/niklasviebig/Desktop/Project_Addax/addax_env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MCD12Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1

  warnings.warn(warning, category=DeprecationWarning)


In [2]:
import ee
import folium
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
# Adjust the 'center', 'zoom', and 'geojson' paths as needed.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Function to get a MODIS Land Cover image for a given year and mask out unwanted classes.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    # Allowed classes for desert areas:
    # 6: Closed Shrublands
    # 7: Open Shrublands
    # 8: Woody Savannas
    # 9: Savannas
    # 10: Grasslands
    # 11: Permanent Wetlands
    # 12: Croplands
    # 13: Urban and Built-up Lands
    # 14: Cropland/Natural Vegetation Mosaics
    # 16: Barren
    # 17: Water Bodies
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    # Build a mask that is True for allowed classes.
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters for the MODIS land cover classification (desert area).
# Note: The data still use the original class numbers, so we set min=6 and max=17,
# and supply a palette only for the allowed classes.
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044',  # Class 6: Closed Shrublands
        'dcd159',  # Class 7: Open Shrublands
        'dade48',  # Class 8: Woody Savannas
        'fbff13',  # Class 9: Savannas
        'b6ff05',  # Class 10: Grasslands
        '27ff87',  # Class 11: Permanent Wetlands
        'c24f44',  # Class 12: Croplands
        'a5a5a5',  # Class 13: Urban and Built-up Lands
        'ff6d4c',  # Class 14: Cropland/Natural Vegetation Mosaics
        'f9ffa4',  # Class 16: Barren
        '1c0dff'   # Class 17: Water Bodies
    ]
}

# Helper function to add Earth Engine layers to a Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Function to add a legend with just the class names.
def add_legend(map_object):
    legend_html = '''
     <div style="
         position: fixed; 
         bottom: 50px; left: 50px; 
         width: 260px; 
         border:2px solid grey; 
         z-index:9999; 
         font-size:12px;
         background-color:white;
         opacity: 0.8;
         padding: 10px;
         ">
         <b>Land Cover Legend</b><br>
         <i style="background:#c6b044; width:18px; height:18px; float:left; margin-right:8px;"></i>Closed Shrublands<br>
         <i style="background:#dcd159; width:18px; height:18px; float:left; margin-right:8px;"></i>Open Shrublands<br>
         <i style="background:#dade48; width:18px; height:18px; float:left; margin-right:8px;"></i>Woody Savannas<br>
         <i style="background:#fbff13; width:18px; height:18px; float:left; margin-right:8px;"></i>Savannas<br>
         <i style="background:#b6ff05; width:18px; height:18px; float:left; margin-right:8px;"></i>Grasslands<br>
         <i style="background:#27ff87; width:18px; height:18px; float:left; margin-right:8px;"></i>Permanent Wetlands<br>
         <i style="background:#c24f44; width:18px; height:18px; float:left; margin-right:8px;"></i>Croplands<br>
         <i style="background:#a5a5a5; width:18px; height:18px; float:left; margin-right:8px;"></i>Urban and Built-up Lands<br>
         <i style="background:#ff6d4c; width:18px; height:18px; float:left; margin-right:8px;"></i>Cropland/Natural Vegetation Mosaics<br>
         <i style="background:#f9ffa4; width:18px; height:18px; float:left; margin-right:8px;"></i>Barren<br>
         <i style="background:#1c0dff; width:18px; height:18px; float:left; margin-right:8px;"></i>Water Bodies<br>
     </div>
    '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

# Create ipywidgets for country selection and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Output widget for displaying the map.
output = widgets.Output()

# Function to update the map based on the selected country and year.
def update_map(change):
    # Get current selections.
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create a new Folium map centered on the selected country.
    new_map = folium.Map(location=center, zoom_start=zoom)
    folium.TileLayer('OpenStreetMap').add_to(new_map)
    
    # Add the GeoJSON layer for the country's boundary.
    folium.GeoJson(
        geojson_path,
        name=f"{country.title()} Boundary"
    ).add_to(new_map)
    
    # Retrieve the MODIS image for the selected year (with masking applied).
    image = get_modis_image(year)
    new_map.add_ee_layer(image, vis_params, 'MODIS Land Cover ' + str(year))
    
    # Add the custom legend.
    add_legend(new_map)
    
    # Add layer control to toggle layers.
    new_map.add_child(folium.LayerControl())
    
    # Display the updated map.
    with output:
        output.clear_output()
        display(new_map)

# Observe changes in both widgets.
country_dropdown.observe(update_map, names='value')
year_slider.observe(update_map, names='value')

# Display the initial map.
update_map({'new': country_dropdown.value})
display(widgets.VBox([country_dropdown, year_slider, output]))

In [3]:
import ee
import json
import time
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps, WidgetControl
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Land cover labels for allowed classes.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS Land Cover image for a given year and mask out unwanted classes.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    # Allowed classes for desert areas.
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044',  # Class 6: Closed Shrublands
        'dcd159',  # Class 7: Open Shrublands
        'dade48',  # Class 8: Woody Savannas
        'fbff13',  # Class 9: Savannas
        'b6ff05',  # Class 10: Grasslands
        '27ff87',  # Class 11: Permanent Wetlands
        'c24f44',  # Class 12: Croplands
        'a5a5a5',  # Class 13: Urban and Built-up Lands
        'ff6d4c',  # Class 14: Cropland/Natural Vegetation Mosaics
        'f9ffa4',  # Class 16: Barren
        '1c0dff'   # Class 17: Water Bodies
    ]
}

# Create ipywidgets for country and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Create an info widget for displaying hover information.
info_widget = widgets.HTML("Hover over the map", placeholder="Hover info")
info_control = WidgetControl(widget=info_widget, position='topright')

# Global variable for debouncing.
last_query_time = 0

def create_map():
    global last_query_time
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create an ipyleaflet Map.
    m = Map(center=center, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)
    
    # Load and add the GeoJSON boundary.
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    geojson_layer = GeoJSON(data=geojson_data, name=f"{country.title()} Boundary",
                             hover_style={'color': 'red', 'fillOpacity': 0.1})
    m.add_layer(geojson_layer)
    
    # Retrieve the MODIS image.
    image = get_modis_image(year)
    map_id_dict = ee.Image(image).getMapId(vis_params)
    ee_tile_url = map_id_dict['tile_fetcher'].url_format
    ee_tile_layer = TileLayer(url=ee_tile_url, name='MODIS Land Cover ' + str(year), opacity=0.7)
    m.add_layer(ee_tile_layer)
    
    m.add_control(LayersControl(position='bottomright'))
    m.add_control(info_control)
    
    # Define an interaction handler to sample the image on mouse move.
    def handle_interaction(**kwargs):
        global last_query_time
        if kwargs.get('type') == 'mousemove':
            coords = kwargs.get('coordinates')
            if coords:
                # Debounce: only query if >0.5 seconds since last query.
                if time.time() - last_query_time < 0.5:
                    return
                last_query_time = time.time()
                point = ee.Geometry.Point(coords)
                try:
                    sample_dict = image.sample(region=point, scale=500, numPixels=1).first().getInfo()
                    if sample_dict and 'properties' in sample_dict and 'LC_Type1' in sample_dict['properties']:
                        lc_val = sample_dict['properties']['LC_Type1']
                        label = modis_labels.get(lc_val, "No data")
                        info_widget.value = f"Coords: {coords[0]:.4f}, {coords[1]:.4f} | Land Cover: {label}"
                    else:
                        info_widget.value = f"Coords: {coords[0]:.4f}, {coords[1]:.4f} | No data"
                except Exception as e:
                    info_widget.value = f"Error sampling data: {e}"
        else:
            info_widget.value = "Hover over the map"
    
    m.on_interaction(handle_interaction)
    return m

# Instead of a function that calls display(), we assign the map to a variable and display it.
m = create_map()
controls = widgets.VBox([country_dropdown, year_slider])
display(controls)
display(m)

Map(center=[20, -10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [4]:
import ee
import json
import time
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps, WidgetControl
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        # Expecting a GeoJSON file that contains individual admin units.
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Allowed MODIS land cover classes and their labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS Land Cover image for a given year and mask out unwanted classes.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    # Allowed classes for desert areas.
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044',  # 6: Closed Shrublands
        'dcd159',  # 7: Open Shrublands
        'dade48',  # 8: Woody Savannas
        'fbff13',  # 9: Savannas
        'b6ff05',  # 10: Grasslands
        '27ff87',  # 11: Permanent Wetlands
        'c24f44',  # 12: Croplands
        'a5a5a5',  # 13: Urban and Built-up Lands
        'ff6d4c',  # 14: Cropland/Natural Vegetation Mosaics
        'f9ffa4',  # 16: Barren
        '1c0dff'   # 17: Water Bodies
    ]
}

# Create ipywidgets for country and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Create HTML widgets to display overall country stats and admin unit stats.
country_stats_widget = widgets.HTML("Country Stats will appear here")
admin_stats_widget = widgets.HTML("Click on an administrative unit for details")

# Create an info widget to display hover coordinates (optional).
info_widget = widgets.HTML("Hover over the map", placeholder="Hover info")
info_control = WidgetControl(widget=info_widget, position='topright')

# Global variable for debouncing hover events.
last_query_time = 0

# Helper function to compute zonal statistics (frequency histogram) and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    # Get the histogram for 'LC_Type1'
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value/total)*100
            except Exception as e:
                continue
    return percentages

# Helper function to format stats as an HTML table.
def format_stats_table(stats_dict, title="Stats"):
    html = f"<h4>{title}</h4><table border='1' style='border-collapse: collapse; font-size:12px;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        html += f"<tr><td>{label}</td><td>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Function to create the ipyleaflet map.
def create_map():
    global last_query_time
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create the ipyleaflet map.
    m = Map(center=center, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)
    
    # Load the GeoJSON for administrative boundaries.
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    geojson_layer = GeoJSON(data=geojson_data, name=f"{country.title()} Units",
                             hover_style={'color': 'red', 'fillOpacity': 0.1})
    m.add_layer(geojson_layer)
    
    # Retrieve the MODIS image.
    image = get_modis_image(year)
    map_id_dict = ee.Image(image).getMapId(vis_params)
    ee_tile_url = map_id_dict['tile_fetcher'].url_format
    ee_tile_layer = TileLayer(url=ee_tile_url, name='MODIS Land Cover ' + str(year), opacity=0.7)
    m.add_layer(ee_tile_layer)
    
    # Add controls.
    m.add_control(LayersControl(position='bottomright'))
    m.add_control(info_control)
    
    # Compute overall country stats.
    # Here, we combine all admin units by unioning the features.
    try:
        # Assume geojson_data is a FeatureCollection.
        features = geojson_data.get('features', [])
        if features:
            # Create a union of all geometries.
            union_geom = ee.Geometry.MultiPolygon([feat['geometry']['coordinates'] for feat in features])
            country_stats = compute_stats(image, union_geom)
            country_stats_widget.value = format_stats_table(country_stats, title="Country Stats")
    except Exception as e:
        country_stats_widget.value = f"Error computing country stats: {e}"
    
    # Define a click handler for admin units.
    def on_feature_click(event, feature, **kwargs):
        try:
            # feature is a dict representing the clicked feature.
            geom = ee.Geometry(feature['geometry'])
            admin_stats = compute_stats(image, geom)
            admin_stats_widget.value = format_stats_table(admin_stats, title="Administrative Unit Stats")
        except Exception as e:
            admin_stats_widget.value = f"Error computing admin stats: {e}"
    
    # Attach the click handler to each feature.
    geojson_layer.on_click(on_feature_click)
    
    # Optional: Update info widget on hover (shows coordinates).
    def handle_interaction(**kwargs):
        global last_query_time
        if kwargs.get('type') == 'mousemove':
            coords = kwargs.get('coordinates')
            if coords:
                if time.time() - last_query_time < 0.5:
                    return
                last_query_time = time.time()
                info_widget.value = f"Coords: {coords[0]:.4f}, {coords[1]:.4f}"
        else:
            info_widget.value = "Hover over the map"
    
    m.on_interaction(handle_interaction)
    return m

# Function to display the map.
def display_map(*args):
    m = create_map()
    display(m)

# Observers: when the country or year changes, update the map.
def refresh_map(change):
    display_map()

country_dropdown.observe(refresh_map, names='value')
year_slider.observe(refresh_map, names='value')

# Display the controls, the overall country stats, and the map.
controls = widgets.VBox([country_dropdown, year_slider, country_stats_widget, admin_stats_widget])
display(controls)
m = create_map()
display(m)

Map(center=[20, -10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [5]:
import ee
import json
import time
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps, WidgetControl
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        # Expecting a GeoJSON file that contains individual admin units.
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Allowed MODIS land cover classes and their labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS Land Cover image for a given year and mask out unwanted classes.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    # Allowed classes for desert areas.
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044',  # 6: Closed Shrublands
        'dcd159',  # 7: Open Shrublands
        'dade48',  # 8: Woody Savannas
        'fbff13',  # 9: Savannas
        'b6ff05',  # 10: Grasslands
        '27ff87',  # 11: Permanent Wetlands
        'c24f44',  # 12: Croplands
        'a5a5a5',  # 13: Urban and Built-up Lands
        'ff6d4c',  # 14: Cropland/Natural Vegetation Mosaics
        'f9ffa4',  # 16: Barren
        '1c0dff'   # 17: Water Bodies
    ]
}

# Create ipywidgets for country and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Create HTML widgets to display overall country stats and admin unit stats.
country_stats_widget = widgets.HTML("Country Stats will appear here")
admin_stats_widget = widgets.HTML("Click on an administrative unit for details")

# Create an info widget to display hover coordinates (optional).
info_widget = widgets.HTML("Hover over the map", placeholder="Hover info")
info_control = WidgetControl(widget=info_widget, position='topright')

# Global variable for debouncing hover events.
last_query_time = 0

# Helper function to compute zonal statistics (frequency histogram) and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value/total)*100
            except Exception as e:
                continue
    return percentages

# Helper function to format stats as an HTML table.
def format_stats_table(stats_dict, title="Stats"):
    html = f"<h4>{title}</h4><table border='1' style='border-collapse: collapse; font-size:12px;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        html += f"<tr><td>{label}</td><td>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Function to create the ipyleaflet map.
def create_map():
    global last_query_time
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create the ipyleaflet map.
    m = Map(center=center, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)
    
    # Load the GeoJSON for administrative boundaries.
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    geojson_layer = GeoJSON(data=geojson_data, name=f"{country.title()} Units",
                             hover_style={'color': 'red', 'fillOpacity': 0.1})
    m.add_layer(geojson_layer)
    
    # Retrieve the MODIS image.
    image = get_modis_image(year)
    map_id_dict = ee.Image(image).getMapId(vis_params)
    ee_tile_url = map_id_dict['tile_fetcher'].url_format
    ee_tile_layer = TileLayer(url=ee_tile_url, name='MODIS Land Cover ' + str(year), opacity=0.7)
    m.add_layer(ee_tile_layer)
    
    # Add controls.
    m.add_control(LayersControl(position='bottomright'))
    m.add_control(info_control)
    
    # Compute overall country stats.
    try:
        # Convert GeoJSON features to an EE FeatureCollection.
        geo_fc = ee.FeatureCollection(geojson_data['features'])
        country_geometry = geo_fc.geometry()
        country_stats = compute_stats(image, country_geometry)
        country_stats_widget.value = format_stats_table(country_stats, title="Country Stats")
    except Exception as e:
        country_stats_widget.value = f"Error computing country stats: {e}"
    
    # Define a click handler for admin units.
    def on_feature_click(event, feature, **kwargs):
        try:
            geom = ee.Geometry(feature['geometry'])
            admin_stats = compute_stats(image, geom)
            admin_stats_widget.value = format_stats_table(admin_stats, title="Administrative Unit Stats")
        except Exception as e:
            admin_stats_widget.value = f"Error computing admin stats: {e}"
    
    geojson_layer.on_click(on_feature_click)
    
    # Optional: Update info widget on hover (shows coordinates).
    def handle_interaction(**kwargs):
        global last_query_time
        if kwargs.get('type') == 'mousemove':
            coords = kwargs.get('coordinates')
            if coords:
                if time.time() - last_query_time < 0.5:
                    return
                last_query_time = time.time()
                info_widget.value = f"Coords: {coords[0]:.4f}, {coords[1]:.4f}"
        else:
            info_widget.value = "Hover over the map"
    
    m.on_interaction(handle_interaction)
    return m

# Function to display the map.
def display_map(*args):
    m = create_map()
    display(m)

# Observers: when the country or year changes, update the map.
def refresh_map(change):
    display_map()

country_dropdown.observe(refresh_map, names='value')
year_slider.observe(refresh_map, names='value')

# Display the controls, the overall country stats, and the map.
controls = widgets.VBox([country_dropdown, year_slider, country_stats_widget, admin_stats_widget])
display(controls)
m = create_map()
display(m)

Map(center=[20, -10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [7]:
import ee
import json
import time
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps, WidgetControl
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        # Expecting a GeoJSON file that contains individual admin units.
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Allowed MODIS land cover classes and their labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS Land Cover image for a given year and mask out unwanted classes.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    # Allowed classes for desert areas.
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044',  # 6: Closed Shrublands
        'dcd159',  # 7: Open Shrublands
        'dade48',  # 8: Woody Savannas
        'fbff13',  # 9: Savannas
        'b6ff05',  # 10: Grasslands
        '27ff87',  # 11: Permanent Wetlands
        'c24f44',  # 12: Croplands
        'a5a5a5',  # 13: Urban and Built-up Lands
        'ff6d4c',  # 14: Cropland/Natural Vegetation Mosaics
        'f9ffa4',  # 16: Barren
        '1c0dff'   # 17: Water Bodies
    ]
}

# Create ipywidgets for country and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Create HTML widgets to display overall country stats and admin unit stats.
country_stats_widget = widgets.HTML("Country Stats will appear here")
admin_stats_widget = widgets.HTML("Click on an administrative unit for details")

# Create an info widget to display hover coordinates (optional).
info_widget = widgets.HTML("Hover over the map", placeholder="Hover info")
info_control = WidgetControl(widget=info_widget, position='topright')

# Global variables for debouncing hover events, the map, tile layer, and selected feature.
last_query_time = 0
current_map = None
current_tile_layer = None
selected_feature = None

# Helper function to compute zonal statistics (frequency histogram) and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value / total) * 100
            except Exception:
                continue
    return percentages

# Helper function to format stats as an HTML table.
def format_stats_table(stats_dict, title="Stats"):
    html = f"<h4>{title}</h4><table border='1' style='border-collapse: collapse; font-size:12px;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        html += f"<tr><td>{label}</td><td>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Click handler for administrative units.
def on_feature_click(event, feature, **kwargs):
    global selected_feature
    selected_feature = feature
    try:
        # Retrieve the unit name from the feature properties.
        unit_name = feature.get("properties", {}).get("name", "Unknown")
        year = year_slider.value
        image = get_modis_image(year)
        admin_stats = compute_stats(image, ee.Geometry(feature["geometry"]))
        admin_stats_widget.value = format_stats_table(admin_stats, title=f"Administrative Unit Stats: {unit_name}")
    except Exception as e:
        admin_stats_widget.value = f"Error computing admin stats: {e}"

# Function to update the map without re-creating the entire map.
def update_map():
    global current_map, current_tile_layer, selected_feature, last_query_time
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create map if not already created.
    if current_map is None:
        current_map = Map(center=center, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)
        current_map.add_control(LayersControl(position='bottomright'))
        current_map.add_control(info_control)
    else:
        # Update center and zoom.
        current_map.center = center
        current_map.zoom = zoom
    
    # Reload GeoJSON layer.
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    # Clean features: preserve "name" if available.
    new_features = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        new_features.append({
            "type": "Feature",
            "geometry": feat["geometry"],
            "properties": {"name": str(name)}
        })
    clean_geojson = {"type": "FeatureCollection", "features": new_features}
    
    # Remove existing GeoJSON layers.
    layers_to_remove = [layer for layer in current_map.layers if isinstance(layer, GeoJSON)]
    for layer in layers_to_remove:
        current_map.remove_layer(layer)
    
    # Add updated GeoJSON layer.
    geojson_layer = GeoJSON(data=clean_geojson, name=f"{country.title()} Units",
                             hover_style={'color': 'red', 'fillOpacity': 0.1})
    geojson_layer.on_click(on_feature_click)
    current_map.add_layer(geojson_layer)
    
    # Retrieve MODIS image.
    image = get_modis_image(year)
    map_id_dict = ee.Image(image).getMapId(vis_params)
    ee_tile_url = map_id_dict['tile_fetcher'].url_format
    
    # Remove existing tile layer if present.
    if current_tile_layer is not None and current_tile_layer in current_map.layers:
        current_map.remove_layer(current_tile_layer)
    current_tile_layer = TileLayer(url=ee_tile_url, name='MODIS Land Cover ' + str(year), opacity=0.7)
    current_map.add_layer(current_tile_layer)
    
    # Compute overall country stats using union of all admin geometries.
    try:
        geo_fc = ee.FeatureCollection(clean_geojson["features"]).map(lambda f: ee.Feature(f.geometry()))
        country_geometry = geo_fc.geometry()
        country_stats = compute_stats(image, country_geometry)
        country_stats_widget.value = format_stats_table(country_stats, title="Country Stats")
    except Exception as e:
        country_stats_widget.value = f"Error computing country stats: {e}"
    
    # If a district was previously selected, update its stats.
    if selected_feature is not None:
        try:
            admin_stats = compute_stats(image, ee.Geometry(selected_feature["geometry"]))
            # Preserve the name from the selected feature.
            unit_name = selected_feature.get("properties", {}).get("name", "Unknown")
            admin_stats_widget.value = format_stats_table(admin_stats, title=f"Administrative Unit Stats: {unit_name}")
        except Exception as e:
            admin_stats_widget.value = f"Error computing admin stats: {e}"
    
    last_query_time = 0

# Observers: when country or year changes, update the map.
def on_controls_change(change):
    update_map()

country_dropdown.observe(on_controls_change, names='value')
year_slider.observe(on_controls_change, names='value')

# Layout: display controls, stats widgets, and the map.
controls = widgets.VBox([country_dropdown, year_slider, country_stats_widget, admin_stats_widget])
display(controls)

# Create and display the map once.
update_map()
display(current_map)

Map(center=[20, -10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [8]:
import ee
import json
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine.
ee.Initialize()

# Dictionary for country parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# MODIS land cover classes and labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS image for a given year (masking to allowed classes).
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters (not used directly in this comparison, but kept here).
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05',
        '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', 'f9ffa4', '1c0dff'
    ]
}

# Helper to compute zonal stats and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value / total) * 100
            except Exception:
                continue
    return percentages

# Helper to format a stats dict as an HTML table.
def format_stats_table(stats_dict, title="Stats"):
    html = f"<h4>{title}</h4><table border='1' style='border-collapse: collapse; font-size:12px;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        html += f"<tr><td>{label}</td><td>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Helper to compute overall country stats for a given country and year.
def compute_country_stats(country, year):
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    # Clean features while preserving "name" if present.
    new_features = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        new_features.append({
            "type": "Feature",
            "geometry": feat["geometry"],
            "properties": {"name": str(name)}
        })
    clean_geojson = {"type": "FeatureCollection", "features": new_features}
    # Create an EE FeatureCollection and union geometries.
    geo_fc = ee.FeatureCollection(clean_geojson["features"]).map(lambda f: ee.Feature(f.geometry()))
    country_geometry = geo_fc.geometry()
    image = get_modis_image(year)
    stats = compute_stats(image, country_geometry)
    return stats

# Helper to compute the difference between two stats dictionaries.
def compute_diff(stats_A, stats_B):
    # Create a set of all categories present in either stats.
    categories = set(stats_A.keys()).union(set(stats_B.keys()))
    diff = {}
    for cat in categories:
        diff[cat] = stats_A.get(cat, 0) - stats_B.get(cat, 0)
    return diff

# --- UI Widgets ---

# Country dropdown.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

# Two separate year sliders.
year_slider_A = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year A:'
)
year_slider_B = widgets.IntSlider(
    value=2010, min=2001, max=2020, step=1, description='Year B:'
)

# Enforce that Year A > Year B.
def enforce_year_order(change):
    if year_slider_A.value <= year_slider_B.value:
        # Adjust B to be one less than A, if possible.
        if year_slider_A.value - 1 >= year_slider_B.min:
            year_slider_B.value = year_slider_A.value - 1
        else:
            # Otherwise, adjust A.
            year_slider_A.value = year_slider_B.value + 1

year_slider_A.observe(enforce_year_order, names='value')
year_slider_B.observe(enforce_year_order, names='value')

# HTML widgets for stats.
stats_A_widget = widgets.HTML("Stats for Year A will appear here")
stats_B_widget = widgets.HTML("Stats for Year B will appear here")
diff_widget = widgets.HTML("Difference (Year A - Year B) will appear here")

# Function to update comparison stats.
def update_comparison(*args):
    country = country_dropdown.value
    year_A = year_slider_A.value
    year_B = year_slider_B.value
    # Compute stats.
    stats_A = compute_country_stats(country, year_A)
    stats_B = compute_country_stats(country, year_B)
    diff = compute_diff(stats_A, stats_B)
    # Update widgets.
    stats_A_widget.value = format_stats_table(stats_A, title=f"Country Stats: {year_A}")
    stats_B_widget.value = format_stats_table(stats_B, title=f"Country Stats: {year_B}")
    diff_widget.value = format_stats_table(diff, title=f"Change (Year {year_A} - Year {year_B})")

# Observe changes.
country_dropdown.observe(update_comparison, names='value')
year_slider_A.observe(update_comparison, names='value')
year_slider_B.observe(update_comparison, names='value')

# Layout: Country dropdown on top; then three columns for Year A, Year B, and Difference.
comparison_box = widgets.HBox([
    widgets.VBox([year_slider_A, stats_A_widget]),
    widgets.VBox([year_slider_B, stats_B_widget]),
    widgets.VBox([diff_widget])
])

# Display the UI.
display(country_dropdown)
display(comparison_box)

# Initial update.
update_comparison()

Dropdown(description='Country:', options=(('Mauritania', 'mauritania'), ('Burkina Faso', 'burkinafaso'), ('Cha…

In [9]:
import ee
import json
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine.
ee.Initialize()

# Dictionary for country parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# MODIS land cover classes and labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS image for a given year (masking to allowed classes).
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# Visualization parameters (not directly used in comparison).
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05',
        '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', 'f9ffa4', '1c0dff'
    ]
}

# Helper to compute zonal stats and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value / total) * 100
            except Exception:
                continue
    return percentages

# Helper to format a stats dict as an HTML table.
# If diff=True, positive differences are highlighted in green and negative in red.
def format_stats_table(stats_dict, title="Stats", diff=False):
    html = f"<h4 style='text-align:center'>{title}</h4>"
    html += "<table border='1' style='border-collapse: collapse; font-size:12px; margin-left:auto; margin-right:auto;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        cell_style = ""
        if diff:
            if perc > 0:
                cell_style = "background-color: #d4f4dd;"  # light green
            elif perc < 0:
                cell_style = "background-color: #f4d4d4;"  # light red
        html += f"<tr><td>{label}</td><td style='{cell_style}'>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Helper to compute overall country stats.
def compute_country_stats(country, year):
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    # Clean features while preserving "name" if present.
    new_features = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        new_features.append({
            "type": "Feature",
            "geometry": feat["geometry"],
            "properties": {"name": str(name)}
        })
    clean_geojson = {"type": "FeatureCollection", "features": new_features}
    # Create an EE FeatureCollection and union geometries.
    geo_fc = ee.FeatureCollection(clean_geojson["features"]).map(lambda f: ee.Feature(f.geometry()))
    country_geometry = geo_fc.geometry()
    image = get_modis_image(year)
    stats = compute_stats(image, country_geometry)
    return stats

# Helper to compute difference between two stats dictionaries.
def compute_diff(stats_A, stats_B):
    categories = set(stats_A.keys()).union(set(stats_B.keys()))
    diff = {}
    for cat in categories:
        diff[cat] = stats_A.get(cat, 0) - stats_B.get(cat, 0)
    return diff

# --- UI Widgets ---

# Country dropdown.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

# Two year sliders.
year_slider_A = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year A:'
)
year_slider_B = widgets.IntSlider(
    value=2010, min=2001, max=2020, step=1, description='Year B:'
)

# Enforce that Year A > Year B.
def enforce_year_order(change):
    if year_slider_A.value <= year_slider_B.value:
        if year_slider_A.value - 1 >= year_slider_B.min:
            year_slider_B.value = year_slider_A.value - 1
        else:
            year_slider_A.value = year_slider_B.value + 1

year_slider_A.observe(enforce_year_order, names='value')
year_slider_B.observe(enforce_year_order, names='value')

# HTML widgets for stats.
stats_A_widget = widgets.HTML("Stats for Year A will appear here")
stats_B_widget = widgets.HTML("Stats for Year B will appear here")
diff_widget = widgets.HTML("Difference (Year A - Year B) will appear here")

# Function to update comparison stats.
def update_comparison(*args):
    country = country_dropdown.value
    year_A = year_slider_A.value
    year_B = year_slider_B.value
    stats_A = compute_country_stats(country, year_A)
    stats_B = compute_country_stats(country, year_B)
    diff = compute_diff(stats_A, stats_B)
    stats_A_widget.value = format_stats_table(stats_A, title=f"Country Stats: {year_A}")
    stats_B_widget.value = format_stats_table(stats_B, title=f"Country Stats: {year_B}")
    diff_widget.value = format_stats_table(diff, title=f"Change (Year {year_A} - Year {year_B})", diff=True)

# Observe changes.
country_dropdown.observe(update_comparison, names='value')
year_slider_A.observe(update_comparison, names='value')
year_slider_B.observe(update_comparison, names='value')

# Layout: Country dropdown on top; then three columns for Year A, Year B, and Difference.
comparison_box = widgets.HBox([
    widgets.VBox([year_slider_A, stats_A_widget]),
    widgets.VBox([year_slider_B, stats_B_widget]),
    widgets.VBox([diff_widget])
])

# Display the UI.
display(country_dropdown)
display(comparison_box)

# Initial update.
update_comparison()

Dropdown(description='Country:', options=(('Mauritania', 'mauritania'), ('Burkina Faso', 'burkinafaso'), ('Cha…

In [10]:
import ee
import json
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine.
ee.Initialize()

# Dictionary for country parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# MODIS land cover classes and labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS image for a given year (masking to allowed classes).
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# (The vis_params remain here for reference.)
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05',
        '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', 'f9ffa4', '1c0dff'
    ]
}

# Helper to compute zonal stats and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value / total) * 100
            except Exception:
                continue
    return percentages

# Helper to format a stats dict as a centered HTML table.
# If diff=True, positive differences are highlighted green and negatives red.
def format_stats_table(stats_dict, title="Stats", diff=False):
    html = f"<h4 style='text-align:center'>{title}</h4>"
    html += "<table border='1' style='border-collapse: collapse; font-size:12px; margin-left:auto; margin-right:auto;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        cell_style = ""
        if diff:
            if perc > 0:
                cell_style = "background-color: #d4f4dd;"
            elif perc < 0:
                cell_style = "background-color: #f4d4d4;"
        html += f"<tr><td>{label}</td><td style='{cell_style}'>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Compute overall country stats.
def compute_country_stats(country, year):
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    # Clean features while preserving "name".
    new_features = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        new_features.append({
            "type": "Feature",
            "geometry": feat["geometry"],
            "properties": {"name": str(name)}
        })
    clean_geojson = {"type": "FeatureCollection", "features": new_features}
    # Union of all admin geometries.
    geo_fc = ee.FeatureCollection(clean_geojson["features"]).map(lambda f: ee.Feature(f.geometry()))
    country_geometry = geo_fc.geometry()
    image = get_modis_image(year)
    stats = compute_stats(image, country_geometry)
    return stats

# Compute stats for a given administrative unit (feature).
def compute_admin_stats(admin_feature, year):
    image = get_modis_image(year)
    geometry = ee.Geometry(admin_feature["geometry"])
    return compute_stats(image, geometry)

# Helper to compute the difference between two stats dictionaries.
def compute_diff(stats_A, stats_B):
    categories = set(stats_A.keys()).union(set(stats_B.keys()))
    diff = {}
    for cat in categories:
        diff[cat] = stats_A.get(cat, 0) - stats_B.get(cat, 0)
    return diff

# --- UI Widgets for Country Comparison ---

# Country dropdown.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

# Two year sliders for country-level comparison.
year_slider_A = widgets.IntSlider(value=2013, min=2001, max=2020, step=1, description='Year A:')
year_slider_B = widgets.IntSlider(value=2010, min=2001, max=2020, step=1, description='Year B:')

# Enforce that Year A > Year B.
def enforce_year_order(change):
    if year_slider_A.value <= year_slider_B.value:
        if year_slider_A.value - 1 >= year_slider_B.min:
            year_slider_B.value = year_slider_A.value - 1
        else:
            year_slider_A.value = year_slider_B.value + 1

year_slider_A.observe(enforce_year_order, names='value')
year_slider_B.observe(enforce_year_order, names='value')

# HTML widgets for country-level stats.
stats_A_widget = widgets.HTML("Country Stats for Year A will appear here")
stats_B_widget = widgets.HTML("Country Stats for Year B will appear here")
diff_widget = widgets.HTML("Difference (Year A - Year B) will appear here")

def update_country_comparison(*args):
    country = country_dropdown.value
    year_A = year_slider_A.value
    year_B = year_slider_B.value
    stats_A = compute_country_stats(country, year_A)
    stats_B = compute_country_stats(country, year_B)
    diff = compute_diff(stats_A, stats_B)
    stats_A_widget.value = format_stats_table(stats_A, title=f"Country Stats: {year_A}")
    stats_B_widget.value = format_stats_table(stats_B, title=f"Country Stats: {year_B}")
    diff_widget.value = format_stats_table(diff, title=f"Change (Year {year_A} - Year {year_B})", diff=True)

country_dropdown.observe(update_country_comparison, names='value')
year_slider_A.observe(update_country_comparison, names='value')
year_slider_B.observe(update_country_comparison, names='value')

country_comparison_box = widgets.VBox([
    widgets.HBox([year_slider_A, year_slider_B]),
    widgets.HBox([stats_A_widget, stats_B_widget, diff_widget])
])

# --- UI Widgets for Administrative Unit Comparison ---

# Administrative unit dropdown (populated dynamically).
admin_dropdown = widgets.Dropdown(options=[], description='Admin Unit:')

# Two year sliders for admin-level comparison.
admin_year_slider_A = widgets.IntSlider(value=2013, min=2001, max=2020, step=1, description='Admin Year A:')
admin_year_slider_B = widgets.IntSlider(value=2010, min=2001, max=2020, step=1, description='Admin Year B:')

def enforce_admin_year_order(change):
    if admin_year_slider_A.value <= admin_year_slider_B.value:
        if admin_year_slider_A.value - 1 >= admin_year_slider_B.min:
            admin_year_slider_B.value = admin_year_slider_A.value - 1
        else:
            admin_year_slider_A.value = admin_year_slider_B.value + 1

admin_year_slider_A.observe(enforce_admin_year_order, names='value')
admin_year_slider_B.observe(enforce_admin_year_order, names='value')

# HTML widgets for admin-level stats.
admin_stats_A_widget = widgets.HTML("Admin Stats for Year A will appear here")
admin_stats_B_widget = widgets.HTML("Admin Stats for Year B will appear here")
admin_diff_widget = widgets.HTML("Change (Year A - Year B) will appear here")

def update_admin_comparison(*args):
    country = country_dropdown.value
    admin_feature = admin_dropdown.value  # This will be a dict representing the feature.
    if admin_feature is None:
        admin_stats_A_widget.value = "Select an administrative unit."
        admin_stats_B_widget.value = ""
        admin_diff_widget.value = ""
        return
    year_A = admin_year_slider_A.value
    year_B = admin_year_slider_B.value
    stats_A = compute_admin_stats(admin_feature, year_A)
    stats_B = compute_admin_stats(admin_feature, year_B)
    diff = compute_diff(stats_A, stats_B)
    # Retrieve the unit name.
    unit_name = admin_feature.get("properties", {}).get("name", "Unknown")
    admin_stats_A_widget.value = format_stats_table(stats_A, title=f"{unit_name} Stats: {year_A}")
    admin_stats_B_widget.value = format_stats_table(stats_B, title=f"{unit_name} Stats: {year_B}")
    admin_diff_widget.value = format_stats_table(diff, title=f"Change ({year_A} - {year_B})", diff=True)

admin_dropdown.observe(update_admin_comparison, names='value')
admin_year_slider_A.observe(update_admin_comparison, names='value')
admin_year_slider_B.observe(update_admin_comparison, names='value')

# Function to update admin_dropdown options based on selected country.
def update_admin_options(*args):
    country = country_dropdown.value
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    options = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        options.append((str(name), feat))
    if options:
        admin_dropdown.options = options
        admin_dropdown.value = options[0][1]
    else:
        admin_dropdown.options = []
        admin_dropdown.value = None

country_dropdown.observe(update_admin_options, names='value')
update_admin_options()  # initial population

admin_comparison_box = widgets.VBox([
    admin_dropdown,
    widgets.HBox([admin_year_slider_A, admin_year_slider_B]),
    widgets.HBox([admin_stats_A_widget, admin_stats_B_widget, admin_diff_widget])
])

# --- Display the UI ---
display(widgets.HTML("<h2>Country-Level Comparison</h2>"))
display(country_dropdown)
display(country_comparison_box)
display(widgets.HTML("<h2>Administrative Unit Comparison</h2>"))
display(admin_comparison_box)

# Initial update.
update_country_comparison()
update_admin_comparison()

HTML(value='<h2>Country-Level Comparison</h2>')

Dropdown(description='Country:', options=(('Mauritania', 'mauritania'), ('Burkina Faso', 'burkinafaso'), ('Cha…

HTML(value='<h2>Administrative Unit Comparison</h2>')

In [11]:
import ee
import json
from ipyleaflet import Map, TileLayer, GeoJSON, LayersControl, basemaps
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine.
ee.Initialize()

# Dictionary for country parameters.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# MODIS land cover classes and labels.
modis_labels = {
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up Lands",
    14: "Cropland/Natural Vegetation Mosaics",
    16: "Barren",
    17: "Water Bodies"
}

# Function to get a MODIS image for a given year (masking to allowed classes).
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    allowed = [6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
    mask = image.eq(allowed[0])
    for val in allowed[1:]:
        mask = mask.Or(image.eq(val))
    return image.updateMask(mask)

# (vis_params remain here for reference.)
vis_params = {
    'min': 6,
    'max': 17,
    'palette': [
        'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05',
        '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', 'f9ffa4', '1c0dff'
    ]
}

# Helper to compute zonal stats and convert counts to percentages.
def compute_stats(image, geometry, scale=500):
    stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=scale,
        maxPixels=1e9
    ).getInfo()
    hist = stats.get('LC_Type1', {})
    total = sum(hist.values())
    percentages = {}
    if total > 0:
        for key, value in hist.items():
            try:
                class_val = int(key)
                label = modis_labels.get(class_val, f"Class {class_val}")
                percentages[label] = (value / total) * 100
            except Exception:
                continue
    return percentages

# Helper to format a stats dict as a centered HTML table.
# If diff=True, positive differences are highlighted green and negatives red.
def format_stats_table(stats_dict, title="Stats", diff=False):
    html = f"<h4 style='text-align:center'>{title}</h4>"
    html += "<table border='1' style='border-collapse: collapse; font-size:12px; margin-left:auto; margin-right:auto;'>"
    html += "<tr><th>Land Cover</th><th>Percentage (%)</th></tr>"
    for label, perc in sorted(stats_dict.items()):
        cell_style = ""
        if diff:
            if perc > 0:
                cell_style = "background-color: #d4f4dd;"
            elif perc < 0:
                cell_style = "background-color: #f4d4d4;"
        html += f"<tr><td>{label}</td><td style='{cell_style}'>{perc:.1f}</td></tr>"
    html += "</table>"
    return html

# Compute overall country stats.
def compute_country_stats(country, year):
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    # Clean features while preserving "name".
    new_features = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        new_features.append({
            "type": "Feature",
            "geometry": feat["geometry"],
            "properties": {"name": str(name)}
        })
    clean_geojson = {"type": "FeatureCollection", "features": new_features}
    # Union of all admin geometries.
    geo_fc = ee.FeatureCollection(clean_geojson["features"]).map(lambda f: ee.Feature(f.geometry()))
    country_geometry = geo_fc.geometry()
    image = get_modis_image(year)
    stats = compute_stats(image, country_geometry)
    return stats

# Compute stats for a given administrative unit (feature).
def compute_admin_stats(admin_feature, year):
    image = get_modis_image(year)
    geometry = ee.Geometry(admin_feature["geometry"])
    return compute_stats(image, geometry)

# Helper to compute the difference between two stats dictionaries.
def compute_diff(stats_A, stats_B):
    categories = set(stats_A.keys()).union(set(stats_B.keys()))
    diff = {}
    for cat in categories:
        diff[cat] = stats_A.get(cat, 0) - stats_B.get(cat, 0)
    return diff

# --- UI Widgets for Country Comparison ---

# Country dropdown.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

# Year dropdowns for country-level comparison.
years = list(range(2001, 2021))
year_dropdown_A = widgets.Dropdown(
    options=years,
    value=2013,
    description='Year A:'
)
year_dropdown_B = widgets.Dropdown(
    options=years,
    value=2010,
    description='Year B:'
)

# Enforce that Year A > Year B.
def enforce_year_order(change):
    if year_dropdown_A.value <= year_dropdown_B.value:
        if year_dropdown_A.value - 1 >= years[0]:
            year_dropdown_B.value = year_dropdown_A.value - 1
        else:
            year_dropdown_A.value = year_dropdown_B.value + 1

year_dropdown_A.observe(enforce_year_order, names='value')
year_dropdown_B.observe(enforce_year_order, names='value')

# HTML widgets for country-level stats.
stats_A_widget = widgets.HTML("Country Stats for Year A will appear here")
stats_B_widget = widgets.HTML("Country Stats for Year B will appear here")
diff_widget = widgets.HTML("Difference (Year A - Year B) will appear here")

def update_country_comparison(*args):
    country = country_dropdown.value
    year_A = year_dropdown_A.value
    year_B = year_dropdown_B.value
    stats_A = compute_country_stats(country, year_A)
    stats_B = compute_country_stats(country, year_B)
    diff = compute_diff(stats_A, stats_B)
    stats_A_widget.value = format_stats_table(stats_A, title=f"Country Stats: {year_A}")
    stats_B_widget.value = format_stats_table(stats_B, title=f"Country Stats: {year_B}")
    diff_widget.value = format_stats_table(diff, title=f"Change (Year {year_A} - Year {year_B})", diff=True)

country_dropdown.observe(update_country_comparison, names='value')
year_dropdown_A.observe(update_country_comparison, names='value')
year_dropdown_B.observe(update_country_comparison, names='value')

country_comparison_box = widgets.VBox([
    widgets.HBox([year_dropdown_A, year_dropdown_B]),
    widgets.HBox([stats_A_widget, stats_B_widget, diff_widget])
])

# --- UI Widgets for Administrative Unit Comparison ---

# Administrative unit dropdown (populated dynamically).
admin_dropdown = widgets.Dropdown(options=[], description='Admin Unit:')

# Year dropdowns for admin-level comparison.
admin_year_dropdown_A = widgets.Dropdown(
    options=years,
    value=2013,
    description='Admin Year A:'
)
admin_year_dropdown_B = widgets.Dropdown(
    options=years,
    value=2010,
    description='Admin Year B:'
)

def enforce_admin_year_order(change):
    if admin_year_dropdown_A.value <= admin_year_dropdown_B.value:
        if admin_year_dropdown_A.value - 1 >= years[0]:
            admin_year_dropdown_B.value = admin_year_dropdown_A.value - 1
        else:
            admin_year_dropdown_A.value = admin_year_dropdown_B.value + 1

admin_year_dropdown_A.observe(enforce_admin_year_order, names='value')
admin_year_dropdown_B.observe(enforce_admin_year_order, names='value')

# HTML widgets for admin-level stats.
admin_stats_A_widget = widgets.HTML("Admin Stats for Year A will appear here")
admin_stats_B_widget = widgets.HTML("Admin Stats for Year B will appear here")
admin_diff_widget = widgets.HTML("Change (Year A - Year B) will appear here")

def update_admin_comparison(*args):
    country = country_dropdown.value
    admin_feature = admin_dropdown.value  # This will be a dict representing the feature.
    if admin_feature is None:
        admin_stats_A_widget.value = "Select an administrative unit."
        admin_stats_B_widget.value = ""
        admin_diff_widget.value = ""
        return
    year_A = admin_year_dropdown_A.value
    year_B = admin_year_dropdown_B.value
    stats_A = compute_admin_stats(admin_feature, year_A)
    stats_B = compute_admin_stats(admin_feature, year_B)
    diff = compute_diff(stats_A, stats_B)
    unit_name = admin_feature.get("properties", {}).get("name", "Unknown")
    admin_stats_A_widget.value = format_stats_table(stats_A, title=f"{unit_name} Stats: {year_A}")
    admin_stats_B_widget.value = format_stats_table(stats_B, title=f"{unit_name} Stats: {year_B}")
    admin_diff_widget.value = format_stats_table(diff, title=f"Change ({year_A} - {year_B})", diff=True)

admin_dropdown.observe(update_admin_comparison, names='value')
admin_year_dropdown_A.observe(update_admin_comparison, names='value')
admin_year_dropdown_B.observe(update_admin_comparison, names='value')

# Function to update admin_dropdown options based on selected country.
def update_admin_options(*args):
    country = country_dropdown.value
    params = countries[country]
    geojson_path = params['geojson']
    with open(geojson_path) as f:
        geojson_data = json.load(f)
    options = []
    for feat in geojson_data.get('features', []):
        props = feat.get("properties", {})
        name = props.get("name", "Unknown")
        options.append((str(name), feat))
    if options:
        admin_dropdown.options = options
        admin_dropdown.value = options[0][1]
    else:
        admin_dropdown.options = []
        admin_dropdown.value = None

country_dropdown.observe(update_admin_options, names='value')
update_admin_options()  # initial population

admin_comparison_box = widgets.VBox([
    admin_dropdown,
    widgets.HBox([admin_year_dropdown_A, admin_year_dropdown_B]),
    widgets.HBox([admin_stats_A_widget, admin_stats_B_widget, admin_diff_widget])
])

# --- Display the UI ---
display(widgets.HTML("<h2>Country-Level Comparison</h2>"))
display(country_dropdown)
display(country_comparison_box)
display(widgets.HTML("<h2>Administrative Unit Comparison</h2>"))
display(admin_comparison_box)

# Initial update.
update_country_comparison()
update_admin_comparison()

HTML(value='<h2>Country-Level Comparison</h2>')

Dropdown(description='Country:', options=(('Mauritania', 'mauritania'), ('Burkina Faso', 'burkinafaso'), ('Cha…

HTML(value='<h2>Administrative Unit Comparison</h2>')